In [ ]:
!pip install -q kaggle

from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d humairmunir/gender-recognizer

In [ ]:
!unzip gender-recognizer.zip -d gender_data

In [ ]:
import shutil
shutil.rmtree("gender_data/dataset/WOMAN/MEN", ignore_errors=True)

In [ ]:
import os
import shutil
import random

# กำหนดเส้นทางของ dataset
dataset_path = "gender_data/dataset"
train_path = "gender_train"
valid_path = "gender_valid"
test_path = "gender_test"  # เพิ่มโฟลเดอร์ Test Data

# ลบข้อมูลเก่า
shutil.rmtree(train_path, ignore_errors=True)
shutil.rmtree(valid_path, ignore_errors=True)
shutil.rmtree(test_path, ignore_errors=True)  # ลบ test_path ด้วย

# สร้างโฟลเดอร์สำหรับ train, validation และ test
for category in ["MEN", "WOMAN"]:
    os.makedirs(os.path.join(train_path, category), exist_ok=True)
    os.makedirs(os.path.join(valid_path, category), exist_ok=True)
    os.makedirs(os.path.join(test_path, category), exist_ok=True)  # สร้างโฟลเดอร์ Test Data

# ฟังก์ชันแบ่งข้อมูลเป็น Train 70%, Validation 15%, Test 15%
def split_data(source, train_dir, valid_dir, test_dir, train_ratio=0.7, valid_ratio=0.15, test_ratio=0.15):
    files = [f for f in os.listdir(source) if os.path.isfile(os.path.join(source, f))]
    random.shuffle(files)

    train_size = int(len(files) * train_ratio)
    valid_size = int(len(files) * valid_ratio)

    train_files = files[:train_size]
    valid_files = files[train_size:train_size + valid_size]
    test_files = files[train_size + valid_size:]  

    # คัดลอกไฟล์ไปยัง Train
    for file in train_files:
        shutil.copy2(os.path.join(source, file), os.path.join(train_dir, file))

    # คัดลอกไฟล์ไปยัง Validation
    for file in valid_files:
        shutil.copy2(os.path.join(source, file), os.path.join(valid_dir, file))

    # คัดลอกไฟล์ไปยัง Test
    for file in test_files:
        shutil.copy2(os.path.join(source, file), os.path.join(test_dir, file))

split_data(
    os.path.join(dataset_path, "MEN"),
    os.path.join(train_path, "MEN"),
    os.path.join(valid_path, "MEN"),
    os.path.join(test_path, "MEN")
)

split_data(
    os.path.join(dataset_path, "WOMAN"),
    os.path.join(train_path, "WOMAN"),
    os.path.join(valid_path, "WOMAN"),
    os.path.join(test_path, "WOMAN")
)

print(f"Train MEN: {len(os.listdir(train_path+'/MEN'))}")
print(f"Train WOMAN: {len(os.listdir(train_path+'/WOMAN'))}")
print(f"Valid MEN: {len(os.listdir(valid_path+'/MEN'))}")
print(f"Valid WOMAN: {len(os.listdir(valid_path+'/WOMAN'))}")
print(f"Test MEN: {len(os.listdir(test_path+'/MEN'))}")
print(f"Test WOMAN: {len(os.listdir(test_path+'/WOMAN'))}")




Train MEN: 452
Train WOMAN: 452
Valid MEN: 96
Valid WOMAN: 96
Test MEN: 98
Test WOMAN: 98


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Load Data
train_data = train_datagen.flow_from_directory(
    train_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode="binary"
)

valid_data = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode="binary"
)



Found 903 images belonging to 2 classes.
Found 192 images belonging to 2 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# สร้างโมเดล CNN
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

# คอมไพล์โมเดล
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# แสดงโครงสร้างโมเดล
model.summary()


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# ใช้ Early Stopping และ ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# เทรนโมเดล
history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=50,
    callbacks=[early_stopping, lr_scheduler]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.5276 - loss: 8.7870 - val_accuracy: 0.5052 - val_loss: 2.2369 - learning_rate: 0.0010
Epoch 2/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.5958 - loss: 2.9207 - val_accuracy: 0.4896 - val_loss: 3.3180 - learning_rate: 0.0010
Epoch 3/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 72s 2s/step - accuracy: 0.6040 - loss: 1.0092 - val_accuracy: 0.4948 - val_loss: 6.0546 - learning_rate: 0.0010
Epoch 4/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 82s 2s/step - accuracy: 0.6038 - loss: 0.9107 - val_accuracy: 0.5260 - val_loss: 3.2147 - learning_rate: 0.0010
Epoch 5/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 81s 2s/step - accuracy: 0.6142 - loss: 0.8099 - val_accuracy: 0.5104 - val_loss: 3.0290 - learning_rate: 5.0000e-04
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.6133 - loss: 0.6700 - val_accuracy: 0.5885 - val_loss: 2.0315 - learning_rate: 5.0000e-04
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.6711 - loss: 0.6212 - val_ac

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(
    test_path,  
    target_size=(128, 128),
    batch_size=32,
    class_mode="binary",
    shuffle=False
)
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc:.4f}")

Found 196 images belonging to 2 classes.
7/7 ━━━━━━━━━━━━━━━━━━━━ 4s 533ms/step - accuracy: 0.8724 - loss: 0.3243
Test Accuracy: 0.8418


In [ ]:
model.save("gender_classification2_model.h5")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ดึงชื่อคลาส
class_labels = list(test_data.class_indices.keys())  # ['MEN', 'WOMAN']

# วนลูปแสดงรูปทั้งหมดจาก Test Set
for test_images, test_labels in test_data:
    for i in range(len(test_images)):  # วนลูปตามจำนวนรูปใน batch
        img = test_images[i]
        true_label = int(test_labels[i])  # ค่าจริง

        # ทำนายผล
        pred_prob = model.predict(np.expand_dims(img, axis=0), verbose=0)[0][0]  # ขยายมิติให้เข้ากับโมเดล
        pred_label = int(pred_prob > 0.5)

        # ดึงชื่อคลาส
        pred_class = class_labels[pred_label]
        true_class = class_labels[true_label]

        # แสดงภาพและผลลัพธ์
        plt.imshow(img)
        plt.axis('off')
        plt.title(f"Predicted: {pred_class}\nTrue: {true_class}\n({pred_prob:.2f})")
        plt.show()






In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from google.colab import files
from PIL import Image

model = load_model("gender_classification2_model.h5")  # เปลี่ยนเป็น path ของโมเดลที่ต้องการใช้

uploaded = files.upload()

for filename in uploaded.keys():
    img = Image.open(filename)
    img = img.convert("RGB")  # เผื่อภาพเป็นขาวดำ จะได้เปลี่ยนเป็น RGB
    img = img.resize((128, 128))  # ปรับขนาดให้ตรงกับโมเดล
    img = np.array(img) / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # เพิ่มมิติให้เข้ากับโมเดล

    pred_prob = model.predict(img, verbose=0)[0][0]
    pred_label = "WOMAN" if pred_prob > 0.5 else "MEN"

    plt.imshow(img.squeeze())
    plt.axis('off')
    plt.title(f"Predicted: {pred_label}\nConfidence: {pred_prob:.2f}")
    plt.show()
